In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
x0 = pd.read_csv("Xtr0_mat50.csv", " ", header=None)
y0 = pd.read_csv("Ytr0.csv", index_col=0)
xt0 = pd.read_csv("Xte0_mat50.csv", " ", header=None)
x0raw = pd.read_csv("Xtr0.csv", " ", header=None)
xt0raw = pd.read_csv("Xte0.csv", " ", header=None)
x0raw.columns = ['N']
xt0raw.columns = ['N']

x1 = pd.read_csv("Xtr1_mat50.csv", " ", header=None)
y1 = pd.read_csv("Ytr1.csv", index_col=0)
xt1 = pd.read_csv("Xte1_mat50.csv", " ", header=None)
x1raw = pd.read_csv("Xtr1.csv", " ", header=None)
xt1raw = pd.read_csv("Xte1.csv", " ", header=None)
x1raw.columns = ['N']
xt1raw.columns = ['N']

x2 = pd.read_csv("Xtr2_mat50.csv", " ", header=None)
y2 = pd.read_csv("Ytr2.csv", index_col=0)
xt2 = pd.read_csv("Xte2_mat50.csv", " ", header=None)
x2raw = pd.read_csv("Xtr2.csv", " ", header=None)
xt2raw = pd.read_csv("Xte2.csv", " ", header=None)
x2raw.columns = ['N']
xt2raw.columns = ['N']

create features

In [19]:
nuc_3 = []
for i in ['A', 'T', 'C', 'G']:
    for j in ['A', 'T', 'C', 'G']:
        for k in ['A', 'T', 'C', 'G']:
            for l in ['A', 'T', 'C', 'G']:
                for m in ['A', 'T', 'C', 'G']:
                    nuc_3.append(i+j+k+l+m)
                    
for i in nuc_3:
    x0raw[i] = 1.*x0raw.N.apply(lambda x: i in x)
    x1raw[i] = 1.*x1raw.N.apply(lambda x: i in x)
    x2raw[i] = 1.*x2raw.N.apply(lambda x: i in x)
    xt0raw[i] = 1.*xt0raw.N.apply(lambda x: i in x)
    xt1raw[i] = 1.*xt1raw.N.apply(lambda x: i in x)
    xt2raw[i] = 1.*xt2raw.N.apply(lambda x: i in x)

x0raw = x0raw.drop("N", axis = 1)
x1raw = x1raw.drop("N", axis = 1)
x2raw = x2raw.drop("N", axis = 1)
xt0raw = xt0raw.drop("N", axis = 1)
xt1raw = xt1raw.drop("N", axis = 1)
xt2raw = xt2raw.drop("N", axis = 1)

define logistic regression

In [20]:
from random import *

def train_logistic(x, y, n=0.003, T=100000, lamb = 0.1):
    w0 = 0
    t = 0
    m = x.shape[0]
    w = np.zeros(x.shape[1])
    y = np.array(y.iloc[:,0])
    x = np.array(x)
    L = []
    while t<T:
        hw = w0 + np.dot(x,w)
        hw = 1/(1+np.exp(-hw))
        grad = np.dot(y-hw,x) - lamb*w
        w = w + n*grad
        w0 = w0 + n*np.sum(y-hw) - n*lamb
        if np.linalg.norm(grad)<0.1: break
        t+=1
    return(w, w0)

def pred_logistic(x, w, w0):
    hw = w0 + w.dot(x.T)
    hw = 1/(1+np.exp(-hw))
#     hw = 1.*(hw > 0.5)
    return(hw)

train

In [21]:
w_0, w0_0 = train_logistic(x0raw, y0, n=0.0001, T= 400, lamb=49)

In [22]:
w_1, w0_1 = train_logistic(x1raw, y1, n=0.00001, T= 1000, lamb=34)

In [23]:
w_2, w0_2 = train_logistic(x2raw, y2, n=0.0001, T= 100, lamb=34)

predict on test set

In [24]:
pred0 = pred_logistic(xt0raw, w_0, w0_0)
pred1 = pred_logistic(xt1raw, w_1, w0_1)
pred2 = pred_logistic(xt2raw, w_2, w0_2)

transform to the appropriate submission format

In [25]:
col2 = np.append(np.append(pred0, pred1), pred2)
col1 = range(0, 3000)
preds = pd.DataFrame({'Id': col1, 'Bound': col2})

In [26]:
sub_noleakage = preds.copy()
sub_noleakage.Bound = 1*(sub_noleakage.Bound > 0.5)

save without the trick

In [27]:
sub_noleakage.to_csv('submission_final_no_leakage4.csv', sep=',', encoding='utf-8',index=False)

In [28]:
x0te = pd.read_table("Xte0.csv", delimiter=' ',header=None)
x1te = pd.read_table("Xte1.csv", delimiter=' ',header=None)
x2te = pd.read_table("Xte2.csv", delimiter=' ',header=None)

apply the trick explained in the report

In [ ]:
xte=x0te.append(x1te).append(x2te)
xte = xte.reset_index(drop=True)
xte.columns = ['V1']
xte['Acount'] = xte['V1'].apply(lambda x: x.count("A"))
xte['Gcount'] = xte['V1'].apply(lambda x: x.count("G"))
xte['Ccount'] = xte['V1'].apply(lambda x: x.count("C"))
xte['Tcount'] = xte['V1'].apply(lambda x: x.count("T"))
xte = pd.concat([xte,preds['Bound']], axis=1, join_axes=[xte.index])
uniques=xte[["Acount", "Gcount", "Ccount", "Tcount"]].drop_duplicates()
xte['new_pred'] = np.nan

In [ ]:
for i in range(0,uniques.shape[0]):
    temp = xte[(xte['Acount'] == uniques.iloc[i,0])& (xte['Gcount'] == uniques.iloc[i,1]) & (xte['Ccount'] == uniques.iloc[i,2]) & (xte['Tcount'] == uniques.iloc[i,3])]
    nr = temp.shape[0]
    zero_vals = temp.sort_values(by=['Bound']).iloc[0:int(nr/2),5]
    temp.loc[temp.isin(zero_vals)['Bound'],'new_pred'] = 0
    temp.loc[(~(temp.isin(zero_vals)))['Bound'],'new_pred'] = 1
    xte.loc[(xte['Acount'] == uniques.iloc[i,0])& (xte['Gcount'] == uniques.iloc[i,1]) & (xte['Ccount'] == uniques.iloc[i,2]) & (xte['Tcount'] == uniques.iloc[i,3]),'new_pred'] = temp['new_pred']

In [ ]:
xte['Id'] = [x for x in  range(0,xte.shape[0])]
sub = xte.loc[:,('new_pred','Id')]
sub.columns = ['Bound', 'Id']
sub.Bound = sub.Bound.astype(int)
sub.to_csv('submission_final_leakage4.csv', sep=',', encoding='utf-8',index=False)